In [15]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd


# Scrap Team Lists

In [25]:
def get_wiki_url(html, year, soup):
    # Create a regular expression to match the target URLs.
    url_regex = rf'"http:\/\/{year}(.*)\.igem\.org\/(.*)"'
    # Find all matching URLs in the HTML.
    urls = re.findall(url_regex, html)
    if len(urls) == 0:
        # Create a regular expression to match the target URLs.
        url_regex = rf'"https:\/\/{year}(.*)\.igem\.org\/(.*)"'
        # Find all matching URLs in the HTML.
        urls = re.findall(url_regex, html)
        if len(urls) > 0:
            wiki = f"https://{year}{urls[0][0]}.igem.org/{urls[0][1]}"
            wiki = wiki.split('">Wiki<')[0]
        elif len(urls) == 0:
            # Create a regular expression to match the target URLs.
            url_regex = rf'"https:\/\/{year}\.igem\.wiki\/(.*)"'
            # Find all matching URLs in the HTML.
            urls = re.findall(url_regex, html)
            if len(urls) > 0:
                wiki = f"https://{year}.igem.wiki/{urls[0]}"
                wiki = wiki.split('">Wiki<')[0]
            else: 
                wiki = None
    else:
        wiki = f"https://{year}{urls[0][0]}.igem.org/{urls[0][1]}"
        wiki = wiki.split('">Wiki<')[0]
    # if soup.find('a', href=re.compile('wiki')) is not None:
    #     wiki = soup.find('a', href=re.compile('wiki')).get('href')
    # elif soup.find('a', href=re.compile('org')) is not None:
    #     wiki = soup.find('a', href=re.compile('igem')).get('href')
    # else:
    #     wiki = None
    return(wiki)
def extract_each_row(row, year):
    # Create a BeautifulSoup object
    soup = BeautifulSoup(row, 'html.parser')

    # Extract the team name
    team_name = soup.find('a', class_='team_name').get_text()
    link_team = soup.find('a', class_='team_name').get('href')
    # wiki_url = soup.find('a', href=True).get('href')
    wiki_url = get_wiki_url(str(row), year, soup)

    # extract other information
    infos = []
    for a in row.split('<td data-order='):
        infos.append(a.split('>')[-1])
    region = infos[2]
    country = infos[3]
    track = infos[4]
    kind = infos[5]
    section = infos[6]
    application = infos[7]
    return(team_name, link_team, wiki_url, region, country, track, kind, section, application)

In [26]:
teams = []
links = []
wikis = []
regions = []
countries = []
tracks = []
kinds = []
sections = []
apps = []
years = []
year_range = [2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]


for year in year_range:
    url = 'https://old.igem.org/Team_List?year=' + str(year)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    all = re.split(r"<tr>", str(soup.table.tbody.tr))[1:]
    print(year)
    for i in range(len(all)):
        team_name, link_team, wiki_url, region, country, track, kind, section, application = extract_each_row(all[i], year)
        teams.append(team_name)
        links.append(link_team)
        wikis.append(wiki_url)
        regions.append(region)
        countries.append(country)
        tracks.append(track)
        kinds.append(kind)
        sections.append(section)
        apps.append(application)
        years.append(year)

2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023


In [27]:
# Create a DataFrame
df = pd.DataFrame({
    'Team name': teams,
    'Profile team': links,
    'Wiki URL': wikis,
    'Region': regions,
    'Country': countries,
    'Track': tracks,
    'Kind': kinds,
    'Section': sections,
    'Application': apps,
    'Year' : years
})


# Print the DataFrame
df

,Team name,Profile team,Wiki URL,Region,Country,Track,Kind,Section,Application,Year
0,Alberta_NINT,https://old.igem.org/Team.cgi?team_id=102,https://2008.igem.org/Team:Alberta_NINT,North America,Canada,,Collegiate,iGEM,Accepted,2008
1,Bay_Area_RSI,https://old.igem.org/Team.cgi?team_id=80,https://2008.igem.org/Team:Bay_Area_RSI,North America,United States,,Collegiate,iGEM,Accepted,2008
2,BCCS-Bristol,https://old.igem.org/Team.cgi?team_id=135,https://2008.igem.org/Team:BCCS-Bristol,Europe,United Kingdom,,Collegiate,iGEM,Accepted,2008
3,Beijing_Normal,https://old.igem.org/Team.cgi?team_id=78,https://2008.igem.org/Team:Beijing_Normal,Asia,China,,Collegiate,iGEM,Accepted,2008
4,Bologna,https://old.igem.org/Team.cgi?team_id=79,https://2008.igem.org/Team:Bologna,Europe,Italy,,Collegiate,iGEM,Accepted,2008
...,...,...,...,...,...,...,...,...,...,...
4227,Zhejiang-United,https://old.igem.org/Team.cgi?team_id=4876,https://2023.igem.wiki/zhejiang-united,Asia,China,High School,Commercial,hs,Accepted,2023
4228,ZJFH-Nanjing,https://old.igem.org/Team.cgi?team_id=4941,https://2023.igem.wiki/zjfh-nanjing,Asia,China,High School,Commercial,hs,Accepted,2023
4229,ZJU-China,https://old.igem.org/Team.cgi?team_id=4628,https://2023.igem.wiki/zju-china,Asia,China,Agriculture,Collegiate,under,Accepted,2023
4230,ZJUintl-China,https://old.igem.org/Team.cgi?team_id=4776,https://2023.igem.wiki/zjuintl-china,Asia,China,Therapeutics,Collegiate,under,Accepted,2023


In [28]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4232 entries, 0 to 4231
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Team name     4232 non-null   object
 1   Profile team  4232 non-null   object
 2   Wiki URL      4232 non-null   object
 3   Region        4232 non-null   object
 4   Country       4232 non-null   object
 5   Track         4232 non-null   object
 6   Kind          4232 non-null   object
 7   Section       4232 non-null   object
 8   Application   4232 non-null   object
 9   Year          4232 non-null   int64 
dtypes: int64(1), object(9)
memory usage: 330.8+ KB


In [34]:
# Print the selected rows
len(df[df.isnull().any(axis=1)])

0

In [30]:
df.to_csv('team_lists_all_years.csv', index=False)

# Scrap Team Profile

In [54]:
# url = 'https://old.igem.org/Team.cgi?team_id=102' #2008
# url = 'https://old.igem.org/Team.cgi?team_id=385' #2010
# url = 'https://old.igem.org/Team.cgi?team_id=1161' #2013
# url = 'https://old.igem.org/Team.cgi?team_id=2815' #2018
# url = 'https://old.igem.org/Team.cgi?team_id=3743' # 2021
url = 'https://old.igem.org/Team.cgi?team_id=4769' #2023


In [83]:
def get_title_abstract(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    content = soup.findAll(id = 'table_abstract')[0].tr.td.get_text().split('\n')
    title = content[0]
    abstract = " ".join(content[1:])
    return(title, abstract)

In [55]:
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
content = soup.findAll(id = 'table_abstract')[0].tr.td.get_text().split('\n')
title = content[0]
abstract = " ".join(content[1:])


In [56]:
print(soup.prettify())

<!DOCTYPE html PUBLIC '-//W3C//DTD XHTML 1.0 Transitional//EN' 'http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd'>
<html dir="ltr" lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="iGEM 2023 Team Information" name="keywords"/>
  <link href="/favicon.ico" rel="shortcut icon"/>
  <link href="/wiki/opensearch_desc.php" rel="search" title="igem.org (English)" type="application/opensearchdescription+xml"/>
  <link href="/wiki/index.php?title=iGEM 2023 Team Information&amp;action=creativecommons" rel="meta" title="Creative Commons" type="application/rdf+xml"/>
  <link href="http://creativecommons.org/licenses/by-sa/3.0/" rel="copyright"/>
  <link href="http://2023.igem.org/wiki/index.php?title=Special:Recentchanges&amp;feed=rss" rel="alternate" title="2023.igem.org RSS Feed" type="application/rss+xml"/>
  <link href="http://2023.igem.org/wiki/index.php?title=Special:Recentchange

In [142]:
import re
res = soup.findAll('table', id = 'table_roster')[2].td.td.a.get_text()
# re.split('[A-Z]', res)
res

'apnp2'

In [78]:
soup.findAll('td')[22]

<td><img src="https://old.igem.org/images/green_check.png"/> Team name has been accepted, thanks.<tr><td><img src="https://old.igem.org/images/green_check.png"/> Shipping address has been entered, thanks.
                 <a href="Shipping.cgi?group_id=5377"><i> View shipping address.</i></a><tr><td><img src="https://old.igem.org/images/green_check.png"/> Your registration fee has been received. Thank you.
                 <a href="http://competition.igem.org/registration/fees"><i> More...</i></a><table_break>
</table_break></td></tr></td></tr></td>

In [86]:
get_title_abstract(url)

('-- Not provided yet --', '-- No abstract provided yet -- ')

In [87]:
titles = []
abstracts = []
for i in df['Profile team']:
    title, abs = get_title_abstract(i)
    titles.append(title)
    abstracts.append(abs)

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

# Scrap Wiki

In [37]:
url = 'https://2022.igem.wiki/tu-eindhoven/engineering'
url = 'https://2022.igem.wiki/nmu-china/'
url = 'https://2023.igem.wiki/ugm-indonesia/engineering/'

r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [28]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <!-- Required meta tags -->
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://static.igem.wiki/common/icons/favicons/igem-2022.svg" rel="shortcut icon"/>
  <link href="https://creativecommons.org/licenses/by/4.0/" rel="license"/>
  <!-- Bootstrap CSS -->
  <link href="static/bootstrap.min.css" rel="stylesheet"/>
  <!-- Custom CSS -->
  <link href="static/style.css" rel="stylesheet"/>
  <title>
   Contribution | NMU_China - iGEM 2022
  </title>
 </head>
 <body>
  <!-- Navigation -->
  <script type="text/javascript">
   document.addEventListener('DOMContentLoaded', function () {
    var winHeight = window.innerHeight,
        docHeight = document.documentElement.scrollHeight,
        progressBar = document.querySelector('#content_progress');
    progressBar.max = docHeight - winHeight;
    progressBar.value = window.scrollY;

    document.addEventListener('scroll', function ()

In [45]:
soup.findAll('nav')

[]

In [33]:
soup.findAll('p')



[<p>
 <b>In the body of a cancer patient, tumor cells are like those moles constantly popping out of a whack-a-mole machine. They grow and reproduce, and along the way, they mutate and evolve into completely different individuals. This poses a difficult problem for current cancer treatment.</b>
 </p>,
 <p style="line-height:150%">
 <b>The traditional cellular immune threapy for cancer is CAR-T therapy, which involves loading T cells with a CAR to recognize tumor cells. However, this CAR is like a little hammer, it can only hit one mole at a time, that is, it can only recognize one tumor antigen at a time. It is powerless against various mutated tumour cells.</b>
 </p>,
 <p style="line-height:150%">
 <b>How can we effectively address such a wide variety of tumor cells? This year, we NMU-China successfully designed a giant hammer---CAR Library, through antibody library technology. The CAR library can recognize almost any tumor antigen, like a giant hammer which can hit all the moles simu

In [38]:
navbar = soup.ul.findAll('li')
pages = []
for i in navbar:
    print(i.a.get('href'))
    pages.append(url+str(i.a.get('href')) )

AttributeError: 'NoneType' object has no attribute 'get'

In [36]:
pages

['https://2022.igem.wiki/nmu-china/index.html',
 'https://2022.igem.wiki/nmu-china/#',
 'https://2022.igem.wiki/nmu-china/contribution',
 'https://2022.igem.wiki/nmu-china/description',
 'https://2022.igem.wiki/nmu-china/design',
 'https://2022.igem.wiki/nmu-china/engineering',
 'https://2022.igem.wiki/nmu-china/experiments',
 'https://2022.igem.wiki/nmu-china/implementation',
 'https://2022.igem.wiki/nmu-china/proof-of-concept',
 'https://2022.igem.wiki/nmu-china/results',
 'https://2022.igem.wiki/nmu-china/#',
 'https://2022.igem.wiki/nmu-china/team',
 'https://2022.igem.wiki/nmu-china/attributions',
 'https://2022.igem.wiki/nmu-china/notebook',
 'https://2022.igem.wiki/nmu-china/model',
 'https://2022.igem.wiki/nmu-china/#',
 'https://2022.igem.wiki/nmu-china/part-collection',
 'https://2022.igem.wiki/nmu-china/parts',
 'https://2022.igem.wiki/nmu-china/improve',
 'https://2022.igem.wiki/nmu-china/safety',
 'https://2022.igem.wiki/nmu-china/#',
 'https://2022.igem.wiki/nmu-china/hum

In [22]:
soup.ul.findAll('ul')

[<ul aria-labelledby="navbarDropdown" class="dropdown-menu">
 <li><a class="dropdown-item" href="team">Our team</a></li>
 <li><a class="dropdown-item" href="attributions">Attributions</a></li>
 <li><a class="dropdown-item" href="sponsors">Sponsors</a></li>
 <li><a class="dropdown-item" href="collaborations">Collaborations</a></li>
 <li><a class="dropdown-item" href="partnership">Partnership</a></li>
 <li><a class="dropdown-item" href="awards">Awards</a></li>
 </ul>,
 <ul aria-labelledby="navbarDropdown" class="dropdown-menu">
 <li><a class="dropdown-item" href="description">Description</a></li>
 <li><a class="dropdown-item" href="engineering">Engineering</a></li>
 <li><a class="dropdown-item" href="implementation">Implementation</a></li>
 <li><a class="dropdown-item" href="proof-of-concept">Proof of Concept</a>
 </li>
 <li><a class="dropdown-item" href="entrepreneurship">Entrepreneurship</a>
 </li>
 <li><a class="dropdown-item" href="model">Model</a></li>
 <li><a class="dropdown-item" 

# Scrap Video Presentation Subtitles

In [19]:
url = 'https://video.igem.org/c/2023_presentations/videos?s=1'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')

In [144]:
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="#fff" name="theme-color"/>
  <meta content="PeerTube" property="og:platform"/>
  <!-- Web Manifest file -->
  <link href="/manifest.webmanifest?37bc54a9b64e32daab994b92a70f8ce78b1fd0ec" rel="manifest"/>
  <link href="/client/assets/images/favicon.png?ee49b6d55360ae36ad8f9650a3001802cadb4a25" rel="icon" type="image/png"/>
  <link href="/client/assets/images/icons/icon-144x144.png" rel="apple-touch-icon" sizes="144x144"/>
  <link href="/client/assets/images/icons/icon-192x192.png" rel="apple-touch-icon" sizes="192x192"/>
  <!-- logo background-image -->
  <style type="text/css">
   .icon-logo {
        background-image: url(/client/assets/images/logo.svg?4a4a1a13bec55c32879f06d3f31ef0f45efcd069);
      }
  </style>
  <!-- base url -->
  <base href="/client/en-US/"/>
  <!-- /!\ The following comment is used by the server to p

In [16]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import os


In [8]:
os.listdir('/Users/macbookpro/Documents/GitHub/chromedriver')

['.DS_Store', 'Google Chrome for Testing.app']

In [25]:
# Setup chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless") # Ensure GUI is off
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--start-maximized')

# Set path to chromedriver as per your configuration
# homedir = os.path.expanduser("~")
path = '/Users/macbookpro/Documents/GitHub/project-discovery-dashboard/virtualenv/chromedriver/Google Chrome for Testing.app/Contents/Frameworks/Google Chrome for Testing Framework.framework/Versions/119.0.6045.105/Google Chrome for Testing Framework'
webdriver_service = Service(path)

In [28]:
from time import sleep
from selenium import webdriver

story = 'https://medium.com/dropout-analytics/selenium-and-geckodriver-on-mac-b411dbfe61bc'
story = story + '?source=friends_link&sk=18e2c2f07fbe1f8ae53fef5ad57dbb12'   # 'https://bit.ly/2WaKraO' <- short link

def gecko_test(site_000=story):
    """
    simple overview:
        1) set up webdriver
        2) load this article 
        3) close up shop 
    
    input:
        >> site_000
            > default: url of this article ('friend link')
    """
    # set the driver 
    driver = webdriver.Firefox()

    # load this article 
    driver.get(site_000)
    # and chill a bit
    sleep(7)

    # k, cool. let's bounce. 
    driver.quit()


# make runable 
if __name__ == '__main__':
    # here we go
    gecko_test()